In [1]:
import pandas as pd
import numpy as np

# 讀有中文路徑的 有問題
### 讀取訓練資料

In [2]:
#copy... C:\Users\poduo\Anaconda3_JupyterNB\SC00030\作業\Kaggle_Titanic\DataSet
# 1
# input_train = "C:\\Users\\poduo\\Anaconda3_JupyterNB\\SC00030\\作業\\Kaggle_Titanic\\DataSet\\train.csv"
# input_test = "C:\\Users\\poduo\\Anaconda3_JupyterNB\\SC00030\\作業\\Kaggle_Titanic\\DataSet\\test.csv"

# 2
# r、b、u + "C://Users/poduo/Anaconda3_JupyterNB/SC00030/作業/Kaggle_Titanic/DataSet/train.csv"
# "C://Users/poduo/Anaconda3_JupyterNB/SC00030/作業/Kaggle_Titanic/DataSet/test.csv"

# 3
# train = pd.read_csv('../DataSet/train.csv')
# test = pd.read_csv('../DataSet/test.csv')


train = pd.read_csv('C://Users/poduo/Anaconda3_JupyterNB/SC00030/train.csv')
org_train = train.copy()
test = pd.read_csv('C://Users/poduo/Anaconda3_JupyterNB/SC00030/test.csv')
org_test = test.copy()

# 傳址
full_data = [train, test]

test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [3]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


## Feature Engineering
form https://www.kaggle.com/sinakhorami/titanic-best-working-classifier

In [4]:
print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [5]:
print (train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


In [6]:
# 計算 家庭成員 + 自己
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
#     為什麼這個會修改到 train

print (train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


In [7]:
for dataset in full_data:
    #非獨自一人 0(False，有家庭成員)；獨自一人 1(True，沒有家庭成員)
    dataset['IsAlone'] = 0 
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

print (train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())

   IsAlone  Survived
0        0  0.505650
1        1  0.303538


In [8]:
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.336957


In [68]:
for dataset in full_data:
#     把遺失值的部分 填入 中位數 ； 為什麼這樣可以修改到 train
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean())

   CategoricalFare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


In [73]:
# 將 遺失值 隨機放入 平均值+-標準差範圍的 整數

for dataset in full_data:
    age_avg = dataset['Age'].mean() #30.27259
    age_std = dataset['Age'].std() #14.1812
    age_null_count = dataset['Age'].isnull().sum() #計算NaN的數量
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

train['CategoricalAge'] = pd.cut(train['Age'], 5)

print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean())

C:\Users\poduo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


  CategoricalAge  Survived
0  (-0.08, 16.0]  0.540541
1   (16.0, 32.0]  0.355056
2   (32.0, 48.0]  0.364706
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


對 Embarked 作 One-hot-encoding

對 Sex 作 Label encoding

Pclass 是否要 Label encoding ?

Age 和 Fare 是否要分箱 和 清除 NaN 資料